In [ ]:
import pandas as pd
import numpy as np
from scipy import stats


1. Considerando las variables earn, height y age, vamos a suponer que nuestro dataset
fuera una población completa. (Prepara los datos adecuadamente).

In [ ]:
data = pd.read_csv('/content/earnings.csv')
print(data.head())

   height  weight  male     earn  earnk ethnicity  education  \
0      74   210.0     1  50000.0   50.0     White       16.0   
1      66   125.0     0  60000.0   60.0     White       16.0   
2      64   126.0     0  30000.0   30.0     White       16.0   
3      65   200.0     0  25000.0   25.0     White       17.0   
4      63   110.0     0  50000.0   50.0     Other       16.0   

   mother_education  father_education  walk  exercise  smokenow  tense  angry  \
0              16.0              16.0     3         3       2.0    0.0    0.0   
1              16.0              16.0     6         5       1.0    0.0    0.0   
2              16.0              16.0     8         1       2.0    1.0    1.0   
3              17.0               NaN     8         1       2.0    0.0    0.0   
4              16.0              16.0     5         6       2.0    0.0    0.0   

   age  
0   45  
1   58  
2   29  
3   57  
4   91  


In [ ]:
# Función para realizar las pruebas de hipótesis
def hypothesis_test(sample, mu, alpha, alternative='two-sided'):
    # Calcular estadístico t y p-valor
    t_stat, p_value = stats.ttest_1samp(sample, mu)

    # Para pruebas unilaterales
    if alternative == 'greater':
        p_value = 1 - stats.t.cdf(t_stat, df=len(sample)-1)
    elif alternative == 'less':
        p_value = stats.t.cdf(t_stat, df=len(sample)-1)

    # Determinar si se acepta o rechaza la hipótesis nula
    reject_null = p_value < alpha

    # Generar la respuesta
    if reject_null:
        result = f"Con nivel de confianza del {100*(1-alpha)}%, se rechaza la hipótesis nula."
    else:
        result = f"Con nivel de confianza del {100*(1-alpha)}%, se acepta la hipótesis nula."

    return t_stat, p_value, result

# Hipótesis para las variables 'earn', 'height' y 'age'
hypotheses = [
    ('earn', 20000, 'two-sided'),
    ('height', 55, 'two-sided'),
    ('age', 44, 'two-sided'),
    ('earn', 35000, 'two-sided'),
    ('height', 68, 'two-sided'),
    ('height', 56, 'two-sided'),
    ('earn', 25000, 'greater'),
    ('age', 63, 'greater'),
    ('age', 51, 'greater'),
    ('earn', 32000, 'less'),
    ('height', 70, 'less'),
    ('age', 62, 'less')
]

# Realizamos las pruebas
results = []

for var, mu, alternative in hypotheses:
    for alpha in [0.01, 0.05, 0.10]:  # Pruebas con diferentes niveles de significancia
        sample = data[var].sample(25, random_state=np.random.randint(1, 1000))  # Muestra de tamaño 25
        t_stat, p_value, result = hypothesis_test(sample, mu, alpha, alternative)
        results.append((var, mu, alpha, t_stat, p_value, result))

# Convertir los resultados a un DataFrame
results_df = pd.DataFrame(results, columns=['Variable', 'Mu', 'Alpha', 'T-stat', 'P-value', 'Resultado'])
print(results_df)

# Calcular la media poblacional para cada variable
means = data[['earn', 'height', 'age']].mean()
print("\nMedia poblacional para cada variable:")
print(means)


   Variable     Mu  Alpha     T-stat       P-value  \
0      earn  20000   0.01   1.176671  2.508658e-01   
1      earn  20000   0.05  -1.991406  5.793550e-02   
2      earn  20000   0.10   0.918872  3.673059e-01   
3    height     55   0.01  15.004132  1.079177e-13   
4    height     55   0.05  19.032276  5.502479e-16   
5    height     55   0.10  11.073023  6.477711e-11   
6       age     44   0.01  -0.588633  5.616070e-01   
7       age     44   0.05   0.928940  3.621696e-01   
8       age     44   0.10  -0.674738  5.062908e-01   
9      earn  35000   0.01  -5.836132  5.104879e-06   
10     earn  35000   0.05  -2.216828  3.636101e-02   
11     earn  35000   0.10  -1.096986  2.835327e-01   
12   height     68   0.01  -2.417579  2.358313e-02   
13   height     68   0.05  -0.053858  9.574943e-01   
14   height     68   0.10  -2.614093  1.521149e-02   
15   height     56   0.01  15.589795  4.674247e-14   
16   height     56   0.05  13.611686  8.820108e-13   
17   height     56   0.10  1

2. Crea una función que estime la proporción de hombres en el dataset, considerando
una muestra de 50 individuos, con niveles de significancia de 0,05 y 0,01. Escribe con
palabras tus resultados ejecutar la función e interpreta. Compara con la proporción
real.

In [ ]:
from statsmodels.stats.proportion import proportions_ztest


In [ ]:
# Función para estimar la proporción de hombres y realizar la prueba de hipótesis
def test_male_proportion(data, sample_size=50, alpha_values=[0.05, 0.01]):
    # Calcular la proporción real de hombres en el dataset
    real_proportion = data['male'].mean()

    # Realizar las pruebas de hipótesis para diferentes niveles de significancia
    results = []
    for alpha in alpha_values:
        # Tomar una muestra aleatoria de tamaño sample_size
        sample = data['male'].sample(sample_size, random_state=np.random.randint(1, 1000))
        sample_proportion = sample.mean()

        # Número de hombres en la muestra
        count_men = sample.sum()

        # Realizar la prueba de proporción con la hipótesis nula: H0: p = p0 (proporción en la población)
        p0 = real_proportion  # Hipótesis nula: la proporción en la muestra es igual a la proporción en la población
        z_stat, p_value = proportions_ztest(count_men, sample_size, p0)

        # Determinar si se rechaza la hipótesis nula
        reject_null = p_value < alpha
        result = f"Con nivel de confianza del {100*(1-alpha)}%, {'se rechaza' if reject_null else 'se acepta'} la hipótesis nula."

        # Guardar los resultados
        results.append((alpha, sample_proportion, z_stat, p_value, result))

    return results, real_proportion

# Ejecutar la función
results, real_proportion = test_male_proportion(data)

# Mostrar los resultados
results_df = pd.DataFrame(results, columns=['Alpha', 'Proporción Muestra', 'Z-Estadístico', 'P-Valor', 'Resultado'])
print(results_df)

# Imprimir la proporción real de hombres en el dataset
print("\nProporción real de hombres en el dataset:")
print(f"{real_proportion * 100:.2f}%")

   Alpha  Proporción Muestra  Z-Estadístico   P-Valor  \
0   0.05                0.32      -0.783633  0.433255   
1   0.01                0.44       0.972995  0.330556   

                                           Resultado  
0  Con nivel de confianza del 95.0%, se acepta la...  
1  Con nivel de confianza del 99.0%, se acepta la...  

Proporción real de hombres en el dataset:
37.17%


3. Considerando el dataset como una muestra, verifica si el género (male=1 significa
‘hombre”) influye sobre el sueldo “earn” de las personas. Explica y justifica tu
procedimiento.

In [ ]:
# Filtrar los datos de hombres y mujeres
hombres = data[data['male'] == 1]['earn']
mujeres = data[data['male'] == 0]['earn']

# Realizar la prueba t de Student para muestras independientes
t_stat, p_value = stats.ttest_ind(hombres, mujeres)

# Imprimir los resultados
print(f"Estadístico t: {t_stat}")
print(f"P-valor: {p_value}")

# Definir el nivel de significancia
alpha = 0.05

# Verificar si rechazamos la hipótesis nula
if p_value < alpha:
    result = f"Con un nivel de confianza del {100*(1-alpha)}%, se rechaza la hipótesis nula. Existen diferencias significativas entre los sueldos de hombres y mujeres."
else:
    result = f"Con un nivel de confianza del {100*(1-alpha)}%, no se rechaza la hipótesis nula. No existen diferencias significativas entre los sueldos de hombres y mujeres."

print(result)

Estadístico t: 13.68318837519989
P-valor: 1.2371946179054945e-40
Con un nivel de confianza del 95.0%, se rechaza la hipótesis nula. Existen diferencias significativas entre los sueldos de hombres y mujeres.


En este caso, como el p-valor es menor que 0.05, se rechazaría la hipótesis nula, indicando que existe una diferencia significativa en los sueldos de hombres y mujeres.

